In [1]:
import argparse
import os
import numpy as np
import math
import sys
import torch
import torch.distributions as distributions
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.utils.data as Data
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
from scipy.io import savemat
import scipy.io as sio
import sys
import importlib
import datetime
import itertools
import pickle
import scipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [2]:
data_dir = "data/case30"
case_dir = os.path.join(data_dir)
number = 20000
train_index = 12000
valid_index = 16000
saveFile = "data/case30/AD_TPBNN/S10"

In [6]:
#input data
mat_input = os.path.join(case_dir, 'outputdata_sample.mat')
PQ_injection = np.transpose(scipy.io.loadmat(mat_input)['power_injection'])[0:number,:]
P_injection = PQ_injection.real
Q_injection = PQ_injection.imag

PQ_injection_d = np.concatenate([P_injection, Q_injection], axis=1)  

# print(Q_injection.shape)
pv = [1-1,2-1,13-1,22-1,23-1,27-1]
slack = 1-1
P_injection = np.delete(P_injection,slack,1)
Q_injection = np.delete(Q_injection,pv,1)
# print(Q_injection.shape)
PQ_injection = np.concatenate([P_injection, Q_injection], axis=1)   

# knwon voltage 
mat_input = os.path.join(case_dir, 'voltage_known.mat')
voltage_known = np.transpose(scipy.io.loadmat(mat_input)['voltage_known'])[0:number,:]
PQ_injection_voltage_known  = np.concatenate([PQ_injection, voltage_known], axis=1)   

#topo_info
mat_input = os.path.join(case_dir, 'topo_info.mat')
topo_info = torch.from_numpy(np.transpose(scipy.io.loadmat(mat_input)['topo_info']))
print(topo_info.shape)
#output data
mat_ang = os.path.join(case_dir, 'voltage_real.mat')
mat_mag = os.path.join(case_dir, 'voltage_imag.mat')
voltage_real = np.transpose(scipy.io.loadmat(mat_ang)['voltage_real'])[0:number,:]
voltage_imag = np.transpose(scipy.io.loadmat(mat_mag)['voltage_imag'])[0:number,:]

voltage =  np.concatenate([voltage_real,voltage_imag], axis=1) 

xTrain = torch.from_numpy(PQ_injection_voltage_known[0:train_index,:])
yTrain = torch.from_numpy(voltage[0:train_index,:])
yTrain_d = torch.from_numpy(PQ_injection_d[0:train_index,:])

xValid = torch.from_numpy(PQ_injection_voltage_known[train_index:valid_index,:])
yValid = torch.from_numpy(voltage[train_index:valid_index,:])
yValid_d = torch.from_numpy(PQ_injection_d[train_index:valid_index,:])

xTest = torch.from_numpy(PQ_injection_voltage_known[valid_index:number,:])
yTest = torch.from_numpy(voltage[valid_index:number,:])
yTest_d = torch.from_numpy(PQ_injection_d[valid_index:number,:])

dataset = torch.utils.data.TensorDataset(xTrain, yTrain, yTrain_d)
print(xValid.shape)
print(xTrain.shape)
print(yTrain.shape)
print(xTest.shape)
print(yTrain_d.shape)

torch.Size([30, 30])
torch.Size([4000, 60])
torch.Size([12000, 60])
torch.Size([12000, 60])
torch.Size([4000, 60])
torch.Size([12000, 60])


In [47]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=600, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=32, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.001, help="learning rate")
parser.add_argument("--alpha", type=float, default=50, help="super:unsuper")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--labels_dim", type=int, default=60, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=60, help="size of each image dimension")
parser.add_argument("--hidden_dim", type=int, default=100, help="size of hidden_dim")
opt = parser.parse_args(args=[])
print(opt)

Namespace(alpha=50, batch_size=32, hidden_dim=100, img_size=60, labels_dim=60, lr=0.001, n_cpu=8, n_epochs=600)


In [48]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
print(cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

False
cpu


In [49]:
# Initialize generator and discriminator
time = []
loss_valid = [] 
batch_loss_train = [] 
avg_train_losses = []
batch_loss_en_train = [] 
avg_train_losses_en = []

class Multiply(nn.Module):
    def __init__(self):
        super(Multiply, self).__init__()

        WG_topo1 = torch.randn(30,30) 
        WB_topo1 = torch.randn(30,30) 

        nn.init.kaiming_uniform_(WG_topo1)
        nn.init.kaiming_uniform_(WB_topo1)

        WG_topo = WG_topo1 * topo_info
        WB_topo = WB_topo1 * topo_info

        self.WG = torch.nn.Parameter(WG_topo) 
        self.WB = torch.nn.Parameter(WB_topo)

        self.ones = torch.ones(30,1)

        self.bp_for_optimizer = torch.nn.Parameter(torch.randn(30,))
        self.bq_for_optimizer = torch.nn.Parameter(torch.randn(30,))
        
        #nn.init.kaiming_uniform_(self.WG)
        #nn.init.kaiming_uniform_(self.WB)
        nn.init.uniform_(self.bp_for_optimizer)
        nn.init.uniform_(self.bq_for_optimizer)

    def bilinear(self,out_en):
                  
        bnn1 = torch.bmm(out_en[:,0:30].unsqueeze(2), out_en[:,0:30].unsqueeze(1)) 
        bnn2 = torch.bmm(out_en[:,30:60].unsqueeze(2), out_en[:,30:60].unsqueeze(1)) 
        bnn3 = torch.bmm(out_en[:,0:30].unsqueeze(2), out_en[:,30:60].unsqueeze(1)) 
        bnn4 = torch.bmm(out_en[:,30:60].unsqueeze(2), out_en[:,0:30].unsqueeze(1)) 
        
        return bnn1,bnn2,bnn3,bnn4
        
    def forward(self,out_en):
        
        [bnn1,bnn2,bnn3,bnn4] = self.bilinear(out_en)

        bias_bp = self.bp_for_optimizer.repeat(32, 1)
        bias_bq = self.bq_for_optimizer.repeat(32, 1)

        S1 = (bnn1+bnn2) * self.WG + (bnn4-bnn3) * self.WB
        S2 = (bnn4-bnn3) * self.WG  - (bnn1+bnn2) * self.WB

        out_d1 = torch.squeeze(torch.matmul(S1, self.ones.to(device))) + bias_bp
        out_d2 = torch.squeeze(torch.matmul(S2, self.ones.to(device))) + bias_bq
        return out_d1, out_d2
    
    def evaltest(self, out_en):
        
        [bnn1,bnn2,bnn3,bnn4] = self.bilinear(out_en)
        
        bias_bp_test = self.bp_for_optimizer.repeat(4000, 1)
        bias_bq_test = self.bq_for_optimizer.repeat(4000, 1)
        
        S1_test = (bnn1+bnn2) * self.WG + (bnn4-bnn3) * self.WB
        S2_test = (bnn4-bnn3) * self.WG  - (bnn1+bnn2) * self.WB

        out_d1_test = torch.squeeze(torch.matmul(S1_test, self.ones.to(device))) + bias_bp_test
        out_d2_test = torch.squeeze(torch.matmul(S2_test, self.ones.to(device))) + bias_bq_test
        
        return out_d1_test, out_d2_test
    
class FC(nn.Module):
    def __init__(self, Multiply):
        super(FC, self).__init__()

        def block(in_feat, out_feat):
            layers = [nn.Linear(in_feat, out_feat)]
            layers.append(nn.ReLU(inplace=True))
            return layers
        
        self.model_en = nn.Sequential(
            *block(opt.labels_dim, opt.hidden_dim),
            *block(opt.hidden_dim, opt.hidden_dim),
            nn.Linear(opt.hidden_dim, opt.img_size),
        )
        
        self.model_d = Multiply()

        
    def forward(self, z):
        out_en = self.model_en(z)

        [out_d1, out_d2] = self.model_d(out_en)
        
        out = torch.cat((out_d1, out_d2),1)

        return out_en, out
    
    def evaltest(self, z):
        out_en_test = self.model_en(z)

        [out_d1_test, out_d2_test] = self.model_d.evaltest(out_en_test)
        
        out_test = torch.cat((out_d1_test, out_d2_test),1)

        return out_en_test, out_test

# Initialize generator and discriminator
MLP = FC(Multiply)
L2_loss = nn.MSELoss()
if cuda:
    MLP.cuda()
    L2_loss.cuda()
# Configure data loader
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True,
    pin_memory = True,
)

# Optimizers
optimizer = torch.optim.Adam(MLP.parameters(), lr=opt.lr)
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
# ----------
#  Training
# ----------
batches_done = 0
for epoch in range(opt.n_epochs):

    for i, (xTrain, yTrain, yTrain_d) in enumerate(dataloader):
            # -----------------
            #  Train Generator
            # -----------------
            # Start measuring time
            startTime = datetime.datetime.now()
            
            # Reset gradients
            optimizer.zero_grad()
            
            # Generate a batch of samples
            [yHatTrain_en,yHatTrain_d] = MLP(xTrain.float().to(device)) 
            
            # Compute loss
            loss = opt.alpha * L2_loss(yHatTrain_en,yTrain.float().to(device)) + L2_loss(yHatTrain_d,yTrain_d.float().to(device))

            loss_en = L2_loss(yHatTrain_en,yTrain.float().to(device))
            # Compute gradients
            loss.backward()
            
            # Optimize
            optimizer.step()
            
            # Finish measuring time
            endTime = datetime.datetime.now()
            
            timeElapsed = abs(endTime - startTime).total_seconds()
            
            time.append(timeElapsed)
            
            batch_loss_train.append(loss.item())  
            batch_loss_en_train.append(loss_en.item()) 
            print(
                "[Epoch %d/%d] [Batch %d/%d][G loss: %.8f]"
                % (epoch, opt.n_epochs, batches_done % len(dataloader), len(dataloader), loss.item()))
            #VALIDATION
            if (batches_done % (len(yValid)//opt.batch_size) == 0):

                with torch.no_grad(): 
                    [yHatValid_en,yHatValid_d] = MLP.evaltest(xValid.float().to(device)) 
                    # Compute loss
                    lossValueValid = L2_loss(yHatValid_en, yValid.to(device))
                    loss_valid.append(lossValueValid.item())
                    
                if epoch >= 299:
                    if (epoch == 299):
                        bestScore = lossValueValid
                        torch.save(MLP.state_dict(), saveFile +'Best'+'.ckpt')
                        torch.save(optimizer.state_dict(), saveFile+'Optim'+'Best'+'.ckpt')
                    else:
                        thisValidScore = lossValueValid
                        if thisValidScore < bestScore:
                            bestScore = thisValidScore
                            torch.save(MLP.state_dict(), saveFile + 'Best'+'.ckpt')
                            torch.save(optimizer.state_dict(), saveFile+'Optim'+'Best'+'.ckpt') 
            batches_done += 1
    train_loss = np.average(batch_loss_train)
    avg_train_losses.append(train_loss)
    train_loss_en = np.average(batch_loss_en_train)
    avg_train_losses_en.append(train_loss_en)
    batch_loss_train = []
    batch_loss_en_train = []

In [51]:
torch.save(time, saveFile + 'Time')
torch.save(MLP.state_dict(), saveFile + 'Last'+'.ckpt')
torch.save(optimizer.state_dict(), saveFile+'Optim'+'Last'+'.ckpt') 
np.save(saveFile+'Train',avg_train_losses)
np.save(saveFile+'Valid',loss_valid)

In [ ]:
plt.plot(avg_train_losses_en[100:2000])
plt.ylabel('mean squared loss')
plt.xlabel('epoch')
plt.title('neurons = 200')
plt.show()

In [ ]:
plt.plot(loss_valid[100:9000:4])
plt.ylabel('mean squared loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
MLP.eval()
[yHatTest_en, yHatTest_d] = MLP.evaltest(xTest.float().to(device))
rmse_test = np.mean((yHatTest_en.detach().cpu().numpy()-yTest.detach().numpy())**2)
print(rmse_test)
rmse_test_d = np.mean((yHatTest_d.detach().cpu().numpy()-yTest_d.detach().numpy())**2)
print(rmse_test_d)
rmse_test1 = np.sqrt(np.mean((yHatTest_en.detach().cpu().numpy()-yTest.detach().numpy())**2,0))
print(rmse_test1)
print(np.mean(rmse_test1[0:29]))
print(np.mean(rmse_test1[29:53]))

In [54]:
np.save(saveFile+'rmse_test_last',rmse_test1)
np.save(saveFile+'yHatTest_last',yHatTest_en.detach().cpu().numpy())

In [ ]:
model = FC(Multiply)
model.load_state_dict(torch.load(saveFile + 'Best'+'.ckpt'))
optimizer.load_state_dict(torch.load(saveFile+'Optim'+'Best'+'.ckpt'))
model.eval()
[yHatTest_en, yHatTest_d] = model.evaltest(xTest.float().to(device))
rmse_test = np.mean((yHatTest_en.detach().cpu().numpy()-yTest.detach().numpy())**2)
print(rmse_test)
rmse_test_d = np.mean((yHatTest_d.detach().cpu().numpy()-yTest_d.detach().numpy())**2)
print(rmse_test_d)
rmse_test3 = np.sqrt(np.mean((yHatTest_en.detach().cpu().numpy()-yTest.detach().numpy())**2,0))
print(rmse_test3)
print(np.mean(rmse_test3[0:29]))
print(np.mean(rmse_test3[29:53]))

In [56]:
np.save(saveFile+'rmse_test_best',rmse_test3)
np.save(saveFile+'yHatTest_best',yHatTest_en.detach().cpu().numpy())